In [1]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()
try:
    from . import scrapper
except:
    import scrapper


In [2]:
list_pick_members1 = ["1.1", "1.2", "1.3", "1.4", "1.5", "1.6", "1.7", "1.8", "1.9", "1.10", "1.11", "1.12", "1.14", "1.15"]
# list_pick_members1_name = ["Canada", "Newfoundland and Labrador", "Prince Edward Island", "Nova Scotia", "New Brunswick", "Quebec", "Ontario", "Manitoba", "Saskatchewan", "Alberta", "British Columbia", "Yukon", "Northwest Territories", "Nunavut"]

list_pick_members2 = ["2.2", "2.3"]
# list_pick_members2_name = ["Salaried employees paid a fixed salary", "Employees paid by the hour"]

# joined_pi = zip(list_pick_members1, list_pick_members1_name)
# joined_pi2 = zip(list_pick_members2, list_pick_members2_name)

pid=str(1410020101)
pickMembers1=str(1.3)
pickMembers2=str(2.1)
startMonth="01"
startYear="2022"
endMonth="12"
endYear="2022"
referencePeriods="20220101%2C20221001"

filter_names = ["Geography", "Type of Employee"]


In [3]:
url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ pickMembers1 + '&pickMembers%5B1%5D='+ pickMembers2 + '&cubeTimeFrame.startMonth='+ startMonth + '&cubeTimeFrame.startYear=' + startYear + '&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=' + referencePeriods
filter_names = ["Geography", "Type of employee"]

dataframe = scrapper.simple_scrapper(url, filter_names)

In [4]:
dataframe["geography"].unique()

array(['Prince Edward Island'], dtype=object)

In [5]:
# list1 = [1, 2, 3]
# list2 = ['a', 'b', 'c']

result = [(x, y) for x in list_pick_members1 for y in list_pick_members2]
print(result)

[('1.1', '2.2'), ('1.1', '2.3'), ('1.2', '2.2'), ('1.2', '2.3'), ('1.3', '2.2'), ('1.3', '2.3'), ('1.4', '2.2'), ('1.4', '2.3'), ('1.5', '2.2'), ('1.5', '2.3'), ('1.6', '2.2'), ('1.6', '2.3'), ('1.7', '2.2'), ('1.7', '2.3'), ('1.8', '2.2'), ('1.8', '2.3'), ('1.9', '2.2'), ('1.9', '2.3'), ('1.10', '2.2'), ('1.10', '2.3'), ('1.11', '2.2'), ('1.11', '2.3'), ('1.12', '2.2'), ('1.12', '2.3'), ('1.14', '2.2'), ('1.14', '2.3'), ('1.15', '2.2'), ('1.15', '2.3')]


In [6]:
pid=str(1410020101)
pickMembers1=str(1.3)
pickMembers2=str(2.1)
startMonth="01"
startYear="2022"
endMonth="12"
endYear="2022"
referencePeriods="20220101%2C20221001"

filter_names = ["Geography", "Type of employee"]
    

In [7]:
table_name = "monthly_employment_by_industry"

# Connecting to Planet Scale
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + os.getenv("USERNAME") + ':' + os.getenv("PASSWORD") + '@' + os.getenv("HOST") + '/' + os.getenv("DATABASE") 

for x, y in result:
    url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ x + '&pickMembers%5B1%5D='+ y + '&cubeTimeFrame.startMonth='+ startMonth + '&cubeTimeFrame.startYear=' + startYear + '&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=' + referencePeriods
    df = scrapper.simple_scrapper(url, filter_names)
    engine = create_engine(conn_string, connect_args=ssl_args)
    with engine.begin() as engine:
        df.to_sql(table_name, engine, if_exists='append', index=False)
        print("Data inserted into database")

Data inserted into database


ResourceClosedError: This Connection is closed